In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from ast import literal_eval
train = pd.read_csv('train.csv', encoding='utf-8', converters={'clean_body': literal_eval})
test = pd.read_csv('test.csv', encoding='utf-8', converters={'clean_body': literal_eval})

In [ ]:
train

,Unnamed: 0,category,clean_body
0,43313,4,"[sent, thursday, blocked, mails, hello, collea..."
1,13008,4,"[wednesday, july, pm, mailbox, almost, importa..."
2,8094,4,"[opportunity, code, hi, please, add, opportuni..."
3,14603,4,"[thursday, pm, removal, hello, decommissioned,..."
4,42338,4,"[sent, monday, timecard, hello, hours, set, au..."
...,...,...,...
96521,22245,0,"[sent, wednesday, july, sensible, information,..."
96522,22246,0,"[sent, wednesday, july, sensitive, information..."
96523,22247,0,"[sent, wed, july, sensitive, information, acce..."
96524,22248,0,"[sent, wednesday, july, sensitive, information..."


In [ ]:
test

,Unnamed: 0,body,category,clean_body
0,10166,hello name problem logging tried logging work ...,4,"[hello, name, problem, logging, tried, logging..."
1,18053,friday leaver form leave date location center ...,4,"[friday, leaver, form, leave, date, location, ..."
2,32083,tuesday pm document purchase order requisition...,5,"[tuesday, pm, document, purchase, order, requi..."
3,15529,card hello please reassign badge cleaning she ...,4,"[card, hello, please, reassign, badge, cleanin..."
4,9558,forwarder calendar organizer rights card hello...,4,"[forwarder, calendar, organizer, rights, card,..."
...,...,...,...,...
14560,36997,question about job dear please message bellow ...,4,"[question, job, dear, please, message, bellow,..."
14561,29558,friday pm ad groups rename hello please rename...,4,"[friday, pm, ad, groups, rename, hello, please..."
14562,5856,re updated bandwidth upgrade hello please kind...,6,"[updated, bandwidth, upgrade, hello, please, k..."
14563,14913,wednesday pm questions dear several questions ...,4,"[wednesday, pm, questions, dear, several, ques..."


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tqdm import tqdm

In [ ]:
# Create X_train, X_test, y_train , y_test

X_train = pd.DataFrame(train['clean_body'])
y_train = train['category']

X_test = pd.DataFrame(test['clean_body'])
y_test =test['category']

In [ ]:
X = pd.concat([X_train, X_test], axis = 0)
X

,clean_body
0,"[sent, thursday, blocked, mails, hello, collea..."
1,"[wednesday, july, pm, mailbox, almost, importa..."
2,"[opportunity, code, hi, please, add, opportuni..."
3,"[thursday, pm, removal, hello, decommissioned,..."
4,"[sent, monday, timecard, hello, hours, set, au..."
...,...
14560,"[question, job, dear, please, message, bellow,..."
14561,"[friday, pm, ad, groups, rename, hello, please..."
14562,"[updated, bandwidth, upgrade, hello, please, k..."
14563,"[wednesday, pm, questions, dear, several, ques..."


In [ ]:
Y= pd.concat([y_train, y_test], axis = 0)
Y

0        4
1        4
2        4
3        4
4        4
        ..
14560    4
14561    4
14562    6
14563    4
14564    4
Name: category, Length: 111091, dtype: int64

In [ ]:
embedding_vector = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

FileNotFoundError: ignored

In [ ]:
embedding_matrix = np.zeros((,300))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X['clean_body'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
X = tokenizer.texts_to_sequences(X['clean_body'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)
X

In [ ]:
Y = pd.get_dummies(Y).values
print('Shape of label tensor:', Y.shape)
Y

In [ ]:
X_train = X[:len(X_train)]
X_test = X[len(X_train):]
y_train = Y[:len(y_train)]
y_test = Y[len(y_train):]

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
y_test

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
1509/1509 [==============================] - 1026s 678ms/step - loss: 0.5627 - accuracy: 0.7937 - val_loss: 0.4077 - val_accuracy: 0.8579
Epoch 2/10
1509/1509 [==============================] - 1014s 672ms/step - loss: 0.3488 - accuracy: 0.8777 - val_loss: 0.4154 - val_accuracy: 0.8502
Epoch 3/10
1509/1509 [==============================] - 1012s 671ms/step - loss: 0.2895 - accuracy: 0.8977 - val_loss: 0.4312 - val_accuracy: 0.8474
Epoch 4/10
1509/1509 [==============================] - 996s 660ms/step - loss: 0.2433 - accuracy: 0.9136 - val_loss: 0.4026 - val_accuracy: 0.8587
Epoch 5/10
1509/1509 [==============================] - 998s 661ms/step - loss: 0.2109 - accuracy: 0.9245 - val_loss: 0.3846 - val_accuracy: 0.8641
Epoch 6/10
 545/1509 [=========>....................] - ETA: 10:26 - loss: 0.1791 - accuracy: 0.9360

In [ ]:
#model.save('enter your path')

# to load it 
# from tensorflow import keras
# model = keras.models.load_model('path/to/location')